In [16]:
import torch
import torchvision #what does this do?
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import numpy as np
import math

In [17]:
#check the complete list of transforms, see how to compose multiple transforms

In [22]:
# Dataset and Dataloader
class WineDataset(Dataset):
    def __init__(self,transforms=None):
        # data loading
        xy = np.loadtxt("9.Wine.csv",delimiter=',',dtype=np.float32,skiprows=1) #skipping the header
        self.x = xy[:, 1:] #first column is thr wine class category
        self.y = xy[:, [0]]  #makes it n_samples x 1, column vector

        self.n_samples, self.n_features = xy.shape 
        self.transform = transforms

    def __getitem__(self,index):
        #dataset[0] - helps in indexing data samples
        sample =  self.x[index], self.y[index] #return tensors
        if self.transform :
            sample = self.transform(sample)
        return sample

    def __len__(self):
        #len(dataset)
        return self.n_samples



In [29]:
#custom transform class, dummy example
class ToTensor():
    def __call__(self,sample): #callable object??
        inputs, labels = sample
        return torch.from_numpy(inputs.astype(np.float32)), torch.from_numpy(labels.astype(np.float32))

In [30]:
wsd = WineDataset(transforms=ToTensor())

In [31]:
data = wsd[0]
features,labels = data
print("features :")
print(features)
print(type(features))
print("labels :")
print(labels)
print(type(labels))

features :
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])
<class 'torch.Tensor'>
labels :
tensor([1.])
<class 'torch.Tensor'>


In [42]:
wsd = WineDataset()

data = wsd[0]
features,labels = data
print("features :")
print(features)
print(type(features))
print("labels :")
print(labels)
print(type(labels))

features :
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
<class 'numpy.ndarray'>
labels :
[1.]
<class 'numpy.ndarray'>


In [43]:
wsd = WineDataset(transforms=ToTensor())
dataloader = DataLoader(wsd,batch_size=4,shuffle=True,num_workers=0)

dataiter = iter(dataloader)
data = dataiter.next()
features,labels = data
print("features :")
print(features)
print(type(features))
print("labels :")
print(labels)
print(type(labels))

#we can see that all batches retrived are tensors, i.e, transforms have been applied on them

features :
tensor([[1.1840e+01, 8.9000e-01, 2.5800e+00, 1.8000e+01, 9.4000e+01, 2.2000e+00,
         2.2100e+00, 2.2000e-01, 2.3500e+00, 3.0500e+00, 7.9000e-01, 3.0800e+00,
         5.2000e+02],
        [1.3830e+01, 1.6500e+00, 2.6000e+00, 1.7200e+01, 9.4000e+01, 2.4500e+00,
         2.9900e+00, 2.2000e-01, 2.2900e+00, 5.6000e+00, 1.2400e+00, 3.3700e+00,
         1.2650e+03],
        [1.4200e+01, 1.7600e+00, 2.4500e+00, 1.5200e+01, 1.1200e+02, 3.2700e+00,
         3.3900e+00, 3.4000e-01, 1.9700e+00, 6.7500e+00, 1.0500e+00, 2.8500e+00,
         1.4500e+03],
        [1.2670e+01, 9.8000e-01, 2.2400e+00, 1.8000e+01, 9.9000e+01, 2.2000e+00,
         1.9400e+00, 3.0000e-01, 1.4600e+00, 2.6200e+00, 1.2300e+00, 3.1600e+00,
         4.5000e+02]])
<class 'torch.Tensor'>
labels :
tensor([[2.],
        [1.],
        [1.],
        [2.]])
<class 'torch.Tensor'>


In [44]:
class Multransform():
    def __init__(self,factor):
        self.factor = factor
    def __call__(self,sample):
        inputs, labels = sample
        inputs *= self.factor #we are transforming only the inputs
        return inputs,labels 

In [48]:
compose_trsf = torchvision.transforms.Compose([ToTensor(),Multransform(4)])

In [49]:
wsd = WineDataset(transforms=compose_trsf)

In [50]:
dataloader = DataLoader(wsd,batch_size=4,shuffle=True,num_workers=0)

dataiter = iter(dataloader)
data = dataiter.next()
features,labels = data
print("features :")
print(features)
print(type(features))
print("labels :")
print(labels)
print(type(labels))


features :
tensor([[5.4480e+01, 1.9800e+01, 9.4000e+00, 8.0000e+01, 3.6800e+02, 8.0000e+00,
         3.2000e+00, 1.8800e+00, 4.0800e+00, 1.7600e+01, 3.6400e+00, 8.2000e+00,
         2.2000e+03],
        [5.2920e+01, 1.3200e+01, 9.1200e+00, 7.4000e+01, 3.9200e+02, 7.2000e+00,
         3.3200e+00, 2.4400e+00, 7.4800e+00, 4.2080e+01, 2.2400e+00, 6.0400e+00,
         2.7000e+03],
        [5.1400e+01, 6.4000e+00, 1.0080e+01, 7.1200e+01, 3.8000e+02, 9.9200e+00,
         9.4800e+00, 1.0400e+00, 5.8400e+00, 1.5720e+01, 4.3600e+00, 1.4520e+01,
         4.0600e+03],
        [4.9720e+01, 6.1200e+00, 9.1600e+00, 8.6000e+01, 3.4400e+02, 1.0960e+01,
         1.2600e+01, 1.5600e+00, 7.0800e+00, 1.5760e+01, 2.7600e+00, 1.1360e+01,
         1.4080e+03]])
<class 'torch.Tensor'>
labels :
tensor([[3.],
        [3.],
        [1.],
        [2.]])
<class 'torch.Tensor'>
